In [18]:
import tensorflow as tf
import pandas as pd
import argparse
import numpy as np

In [29]:
df  = pd.read_csv(r'idda2.csv', index_col=0)
df_yeni = pd.read_csv(r'iddaa11302018.csv', index_col=0, decimal=',')
#df['winner_team'] = df['winner_team'].apply(lambda x: 'home' if x == 1 else x).apply(lambda x: 'away' if x==2 else 'draw')

In [30]:
columns = ['hometeam_bid','draw_bid','away_bid']

for colm in columns:
    #df_yeni = df_yeni.drop(df_yeni[df_yeni[colm].str.contains('%')].index)
    df_yeni[colm] = df_yeni[colm].apply(lambda x: float(str(x).replace(',','.')))

AttributeError: 'float' object has no attribute 'replace'

In [46]:
df_yeni = df_yeni[df_yeni['winner_team'].notnull()==False]
df_yeni.head()

,week,code,home,away,firsthalf_home,firsthalf_away,final_home,final_away,hometeam_bid,draw_bid,away_bid,winner_team
2507,18094,101,AtikerKonyaspor,Alanyaspor,NaN,NaN,1.45,3.40,4.55,2,NaN,NaN
2508,18094,102,ÇaykurRizespor,MKEAnkaragücü,NaN,NaN,1.75,3.20,3.20,2,NaN,NaN
2509,18094,164,Adanaspor,Hatayspor,NaN,NaN,2.30,2.80,2.45,2,NaN,NaN
2510,18094,169,ErzgebirgeAue,JahnRegensburg,NaN,NaN,2.60,2.90,2.10,2,NaN,NaN
2511,18094,171,Esbjerg,Aalborg,NaN,NaN,2.80,2.90,2.00,2,NaN,NaN


In [47]:
df_yeni['away_bid'] = df_yeni['hometeam_bid']
df_yeni['draw_bid'] = df_yeni['final_away']
df_yeni['hometeam_bid'] = df_yeni['final_home']
df_yeni.head()

,week,code,home,away,firsthalf_home,firsthalf_away,final_home,final_away,hometeam_bid,draw_bid,away_bid,winner_team
2507,18094,101,AtikerKonyaspor,Alanyaspor,NaN,NaN,1.45,3.40,1.45,3.40,4.55,NaN
2508,18094,102,ÇaykurRizespor,MKEAnkaragücü,NaN,NaN,1.75,3.20,1.75,3.20,3.20,NaN
2509,18094,164,Adanaspor,Hatayspor,NaN,NaN,2.30,2.80,2.30,2.80,2.45,NaN
2510,18094,169,ErzgebirgeAue,JahnRegensburg,NaN,NaN,2.60,2.90,2.60,2.90,2.10,NaN
2511,18094,171,Esbjerg,Aalborg,NaN,NaN,2.80,2.90,2.80,2.90,2.00,NaN


In [48]:
li= []

for val in df['home'].values:
    li.append(val)
for val in df['away'].values:
    li.append(val)
for val in df_yeni['home'].values:
    li.append(val)
for val in df_yeni['away'].values:
    li.append(val)

li = list(set(li))

In [49]:
df = df[['home','away','hometeam_bid','draw_bid','away_bid','winner_team']]
df_yeni = df_yeni[['home','away','hometeam_bid','draw_bid','away_bid','winner_team']]

In [50]:

train = df
test =df_yeni
test1 = df_yeni
df.iloc[:5,:]

,home,away,hometeam_bid,draw_bid,away_bid,winner_team
0,Eibar,LasPalmas,1.60,3.1,4.00,2.0
1,StokeCity,Liverpool,2.50,3.1,2.10,2.0
2,Cadiz,CeltaVigo,3.75,3.2,1.60,2.0
3,Everton,ManchesterCity,2.90,3.2,1.85,1.0
4,Genoa,Sampdoria,1.90,2.9,3.00,2.0


In [51]:
test.dtypes

home             object
away             object
hometeam_bid     object
draw_bid         object
away_bid         object
winner_team     float64
dtype: object

In [53]:
train['hometeam_bid']= train['hometeam_bid'].astype(float)
test['hometeam_bid']= test['hometeam_bid'].astype(float)
train['draw_bid']= train['draw_bid'].astype(float)
test['draw_bid']= test['draw_bid'].astype(float)
train['away_bid']= train['away_bid'].astype(float)
test['away_bid']= test['away_bid'].astype(float)
train['winner_team']= train['winner_team'].apply( lambda x: int(x))
#test['winner_team']= test['winner_team'].apply(lambda x: int(x))


In [54]:
test.dtypes

home             object
away             object
hometeam_bid    float64
draw_bid        float64
away_bid        float64
winner_team     float64
dtype: object

In [55]:
train_features, train_label = train, train.pop('winner_team')

test_features, test_label = test , test.pop('winner_team')

,home,away,hometeam_bid,draw_bid,away_bid
0,Eibar,LasPalmas,1.60,3.10,4.00
1,StokeCity,Liverpool,2.50,3.10,2.10
2,Cadiz,CeltaVigo,3.75,3.20,1.60
3,Everton,ManchesterCity,2.90,3.20,1.85
4,Genoa,Sampdoria,1.90,2.90,3.00
5,WiganAthletic,Gillingham,1.80,3.20,3.00
6,RayoVallecano,AtleticoMadrid,5.00,3.50,1.40
7,Napoli,Torino,1.25,4.00,7.00
8,Empoli,Inter,2.90,3.00,1.90
9,VitoriaSetubal,SportingLisbon,5.50,3.60,1.35


In [56]:
homecat = tf.feature_column.categorical_column_with_vocabulary_list(key='home',vocabulary_list=li)
awaycat = tf.feature_column.categorical_column_with_vocabulary_list(key='away', vocabulary_list=li)

my_feature_columns = [
    tf.feature_column.indicator_column(homecat),
    tf.feature_column.indicator_column(awaycat),
    tf.feature_column.numeric_column(key='hometeam_bid'),
    tf.feature_column.numeric_column(key='draw_bid'),
    #tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key='Sex', vocabulary_list=['female','male'])),
    tf.feature_column.numeric_column(key='away_bid')
]

In [61]:
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
        hidden_units=[10, 10],
        n_classes=3 , model_dir='./iddaa_model113018' , config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1) )



INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000254AED5FC88>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 1, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './iddaa_model113018'}


In [62]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [63]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [327]:
#validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(test_set.data,)

In [64]:
classifier.train(
        input_fn=lambda:train_input_fn(train_features, train_label, 100), 
        steps=10000 )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:loss = 110.89708, step = 1
INFO:tensorflow:global_step/sec: 192.092
INFO:tensorflow:loss = 101.243744, step = 101 (0.521 sec)
INFO:tensorflow:Saving checkpoints for 113 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 112.439
INFO:tensorflow:loss = 97.89066, step = 201 (0.890 sec)
INFO:tensorflow:Saving checkpoints for 234 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 110.502
INFO:tensorflow:loss = 95.244316, step = 301 (0.905 sec)
INFO:tensorflow:Saving checkpoints for 355 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 110.849
INFO:tensorflow:loss = 106.64342, step = 401 (0.902 sec)
INFO:tenso

INFO:tensorflow:loss = 76.22917, step = 4701 (0.908 sec)
INFO:tensorflow:Saving checkpoints for 4756 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 88.4596
INFO:tensorflow:loss = 76.650345, step = 4801 (1.129 sec)
INFO:tensorflow:Saving checkpoints for 4828 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 112.44
INFO:tensorflow:loss = 74.36097, step = 4901 (0.890 sec)
INFO:tensorflow:Saving checkpoints for 4955 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 114.68
INFO:tensorflow:loss = 82.54694, step = 5001 (0.872 sec)
INFO:tensorflow:Saving checkpoints for 5084 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 114.477
INFO:tensorflow:loss = 66.150536, step = 5101 (0.873 sec)
INFO:tensorflow:global_step/sec: 216.724
INFO:tensorflow:loss = 80.22675, step = 5201 (0.462 sec)
INFO:tensorflow:Saving checkpoints for 5211 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:Saving checkpoints for 5281 into

INFO:tensorflow:global_step/sec: 63.966
INFO:tensorflow:loss = 69.12779, step = 9401 (1.563 sec)
INFO:tensorflow:global_step/sec: 216.094
INFO:tensorflow:loss = 78.28854, step = 9501 (0.463 sec)
INFO:tensorflow:Saving checkpoints for 9521 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 111.033
INFO:tensorflow:loss = 73.39056, step = 9601 (0.902 sec)
INFO:tensorflow:Saving checkpoints for 9641 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 109.885
INFO:tensorflow:loss = 71.12814, step = 9701 (0.909 sec)
INFO:tensorflow:Saving checkpoints for 9760 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 86.3174
INFO:tensorflow:loss = 82.97875, step = 9801 (1.159 sec)
INFO:tensorflow:Saving checkpoints for 9830 into ./iddaa_model113018\model.ckpt.
INFO:tensorflow:global_step/sec: 114.148
INFO:tensorflow:loss = 68.75016, step = 9901 (0.877 sec)
INFO:tensorflow:Saving checkpoints for 9955 into ./iddaa_model113018\model.ckpt.
INFO:ten

In [65]:
#eval_result = classifier.evaluate(
#input_fn=lambda:eval_input_fn(test_features, test_label, 100))
expected = [0,1,2]
predictions = classifier.predict(
      input_fn=lambda:eval_input_fn(test_features,
                                                labels=None,
                                                batch_size=100))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

#for pred_dict, expec in zip(predictions, expected):
#        class_id = pred_dict['class_ids'][0]
#        probability = pred_dict['probabilities'][class_id]
#
#        print(template.format([0,1][class_id],
#                              100 * probability, expec))
#


In [342]:
#for pred_dict in predictions:
#    class_id = pred_dict['class_ids'][0]
#    probability = pred_dict['probabilities'][class_id]
#    #print(probability)
#    print([0,1,2][class_id])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./iddaa_model10000\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1
1
1
2
1
0
0
1
0
2
2
0
1
0
0
1
1
0
1
0
1
0
2
1
1
1
1
1
0
0
1
1
2
1
1
1
0
2
2
1
2
2
2
1
2
1
2
1
1
2
2
1
1
1
2
1
2
0
1
2
0
1
0
2
0
2
1
0
0
1
0
2
2
2
1
1
1
1
1
1
0
1
2
2
2
0
2
0
1
0
1
2
2
1
2
0
1
1
1
1
0
1
1
0
1
2
1
0
1
0
1
0
0
1
1
1
1
1
2
0
2
2
2
1
0
2
2
1
0
2
1
0
2
0
1
0
0
2
0
1
2
0
1
2
2
2
0
1
2
1
0
1
1
0
2
2
1
1
1
1
1
0
2
0
0
1
1
1
1
1
0
0
0
1
1
0
0
0
1
1
2
1
1
0
2
2
2
0
2
0
2
2
2
1
2
0
1
1
1
2
0
1
0
2
0
0
1
1
1
1
1
0
1
0
1
1
0
2
1
2
0
1
1
1
1
1
0
0
1
1
0
1
2
0
1
1
1
1
0
1
0
0
1
1
1
2
1
1
1
1
0
1
1
2
0
2
1
0
1
1
0
1
0
1
1
0
1
2
1
2
0
1
0
1
1
2
1
1
0
0
0
0
0
0
0
0
2
1
1
2
0
1
0
0
1
0
1
0
1
0
1
1
1
2
2
1
2
2
1
1
2
1
0
0
1
1
2
1
1
2
0
0
1
2
1
1
1
2
0
2
1
1
2
1
2
2
2
0
1
0
2
1
1
0
0
1
2
1
1
1
0
2
1
1
0
1
2
1
1
1
0
2
1
2
1

In [66]:
savedf = pd.DataFrame()
savelist = []
for p in predictions:
    #print(p['class_ids'][0],' ',p['probabilities'][p['class_ids'][0]])
    #savedf = savedf.append([p['class_ids'][0],' ',p['probabilities'][p['class_ids'][0]]])
    savelist.append([p['class_ids'][0],p['probabilities'][p['class_ids'][0]]])
savedf = pd.DataFrame(savelist)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./iddaa_model113018\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [67]:
t1 = pd.DataFrame(test1, index=None).reset_index()
df3 = pd.concat([t1, savedf], axis=1)

In [69]:
df3.to_csv('iddaaresult.csv')